# Section 1: Exploring Data For Creating Evaluation

In [1]:
import pandas as pd
from fastparquet import ParquetFile
import duckdb

In [27]:
""" Initial Load Of Datasets We're Using """
autographs = ParquetFile("datasets/autographs.parquet").to_pandas()
graffitis = ParquetFile("datasets/graffiti.parquet").to_pandas()
stickers = ParquetFile("datasets/sticker.parquet").to_pandas()
matches = ParquetFile("datasets/matches_edit.parquet").to_pandas()

In [ ]:
""" Fixing Tournament Dates And Names In Matches """
query = f"""
    SELECT 
        strptime(regexp_replace(date, '([0-9]+)(st|nd|rd|th)', '\\1'), '%B %d %Y') AS date,
        event,
        winner,
        loser,
        winner-score,
        loser-score
    FROM matches
"""
df = duckdb.query(query).df()

tournament_name_map = {
    "Perfect World Shanghai Major 2024" : "Perfect World Shanghai 2024",
    "IEM Rio Major 2022"                : "IEM Rio 2022",
    "EMS One Katowice 2014"             : "2014 EMS One Katowice",
    "ESL One Cologne 2014"              : "2014 ESL One Cologne",
    "PGL CS2 Major Copenhagen 2024"     : "PGL Copenhagen 2024",
    "ESL One Katowice 2015"             : "2015 ESL One Katowice",
    "DreamHack Winter 2014"             : "2014 DreamHack Winter",
    "MLG Columbus 2016"                 : "2016 MLG Columbus",
    "PGL Major Stockholm 2021"          : "2021 PGL Stockholm",
    "StarLadder Major Berlin 2019"      : "2019 StarLadder Berlin",
    "ELEAGUE Major 2017"                : "2017 ELEAGUE Atlanta",
    "BLAST.tv Paris Major 2023"         : "BLAST.tv Paris 2023",
    "PGL Major Antwerp 2022"            : "2022 PGL Antwerp",
    "FACEIT Major 2018"                 : "2018 FACEIT London",
    "PGL Major Krakow 2017"             : "2017 PGL Krakow",
    "IEM Katowice 2019"                 : "2019 IEM Katowice",
    "DreamHack Open Cluj-Napoca 2015"   : "2015 DreamHack Cluj-Napoca",
    "ELEAGUE Major 2018"                : "2018 ELEAGUE Boston",
    "ESL One Cologne 2015"              : "2015 ESL One Cologne",
    "ESL One Cologne 2016"              : "2016 ESL One Cologne",
}

df["event"] = df["event"].map(tournament_name_map)
# df.to_parquet("matches_edit.parquet")

In [ ]:
""" Ranking Teams In Each Tournament """
query = f"""
    SELECT FIRST(winner) AS first_place, FIRST(loser) AS second_place, event
    FROM matches
    GROUP BY event
"""
duckdb.query(query).df()

In [30]:
""" Getting Tournament Dates And Top 2 Teams"""
query = f"""
    WITH
        top_two AS (
            SELECT
                FIRST(winner) AS first_place, FIRST(loser) AS second_place, event
            FROM matches
            GROUP BY event
        ),
        start_end AS (
            SELECT 
                event, 
                MIN(date) AS start_date, 
                MAX(date) AS end_date
            FROM matches
            GROUP BY event
        )
    SELECT 
        start_end.event, start_date, end_date, first_place, second_place
    FROM start_end
    JOIN top_two ON
        start_end.event = top_two.event
"""

tournaments = duckdb.query(query).df()

In [36]:
""" Getting Sticker Sales During +- 15 Dates Of Tournament """
query = f"""
    SELECT
        item,
        tournament,
        team,
        AVG(price) AS avg_price,
        AVG(volume) AS avg_volume
    FROM stickers
    JOIN tournaments ON
        stickers.tournament = tournaments.event AND
        stickers.date >= (tournaments.start_date - INTERVAL 15 DAY) AND
        stickers.date <= (tournaments.end_date + INTERVAL 15 DAY)
    GROUP BY
        item, tournament, team, stickers.date
    ORDER BY
        tournament, avg_volume DESC
    
"""
duckdb.query(query).df()

,item,tournament,team,avg_price,avg_volume
0,Sticker | iBUYPOWER | DreamHack 2014,2014 DreamHack Winter,iBUYPOWER,0.244,12991.0
1,Sticker | Ninjas in Pyjamas | DreamHack 2014,2014 DreamHack Winter,Ninjas in Pyjamas,0.240,10759.0
2,Sticker | Team LDLC.com | DreamHack 2014,2014 DreamHack Winter,Team LDLC.com,0.240,9132.0
3,Sticker | iBUYPOWER | DreamHack 2014,2014 DreamHack Winter,iBUYPOWER,0.241,8303.0
4,Sticker | iBUYPOWER | DreamHack 2014,2014 DreamHack Winter,iBUYPOWER,0.368,7972.0
...,...,...,...,...,...
176153,Sticker | hallzerk (Holo) | Shanghai 2024,Perfect World Shanghai 2024,Complexity Gaming,2.300,1.0
176154,Sticker | Grim (Gold) | Shanghai 2024,Perfect World Shanghai 2024,Complexity Gaming,7.109,1.0
176155,Sticker | felps (Holo) | Shanghai 2024,Perfect World Shanghai 2024,Imperial Esports,0.038,1.0
176156,Sticker | JDC (Gold) | Shanghai 2024,Perfect World Shanghai 2024,BIG,23.718,1.0
